Problem Statement

The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with calculating the loss associated with a loan defaults. Although the features are anonymized, they have properties relating to real-world features.

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/train.csv")
train

In [ ]:
test = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/test.csv")
test

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv")
submission

Analyse target

In [ ]:
sns.displot(train['loss']);

In [ ]:
train['loss'].describe()

In [ ]:
target = train.loss
train.drop(['loss'], axis=1, inplace=True)
train

Combine

In [ ]:
combi = train.append(test)
combi

In [ ]:
combi.drop(['id'], axis=1, inplace=True)
combi

Check for null values

In [ ]:
combi.isnull().sum().sum()

Normalise

In [ ]:
combi = (combi - combi.min()) / (combi.max() - combi.min())
combi.shape

Define X and y

In [ ]:
length = len(train)

y = target.ravel()
X = combi[: length]
X_test = combi[length :]
y.shape, X.shape, X_test.shape

Generic Univariate Select

In [ ]:
from sklearn.feature_selection import GenericUnivariateSelect, f_regression

transformer = GenericUnivariateSelect(f_regression, mode='k_best', param=10)
X = transformer.fit_transform(X, y)
X_test = transformer.transform(X_test)
X.shape, y.shape, X_test.shape

Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X_test.shape

Select Model

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)
print("Score: ",model.score(X_train, y_train))
print("")
print("Coefgficient: ", model.coef_)
print("")
print("Intercept: ", model.intercept_)

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))
print("")
print("Coefgficient: ", model.coef_)
print("")
print("Intercept: ", model.intercept_)

Evaluate

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_val, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, y_pred)))

Compare

In [ ]:
compare = pd.DataFrame({'actual': y_val, 'predicted': y_pred})
print(compare)

Graph

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_val, y_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred), max(y_val))
p2 = min(min(y_pred), min(y_val))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()


Predict on test set

In [ ]:
prediction = model.predict(X_test)
prediction[prediction < 0] = 0
prediction.shape

Prepare submission

In [ ]:
submission.loss = prediction
submission

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission
